In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import operator
import re
from nltk import FreqDist,pos_tag
import codecs
import tokenize
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist,pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from sklearn.naive_bayes import MultinomialNB
from collections import defaultdict

from bs4 import BeautifulSoup

In [2]:
with codecs.open("AnimeReviews.csv") as f:
    df = pd.read_csv(f)

/Users/garygao/Desktop/CS4300_Flask_template/venv/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,8,30,39,40,41,42,43,44,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index([u'anime_id', u'anime_title', u'anime_english_title',
       u'anime_japanese_title', u'anime_synonyms', u'anime_image_url',
       u'anime_genres', u'anime_synopsis', u'anime_rating_value',
       u'anime_rating_count', u'anime_rating', u'anime_ranked',
       u'anime_popularity', u'anime_favorites', u'anime_members',
       u'anime_number_of_episodes', u'anime_type', u'anime_source',
       u'anime_background', u'anime_premiered', u'anime_status',
       u'anime_aired', u'anime_broadcast', u'anime_producers',
       u'anime_licensors', u'anime_studios', u'anime_duration',
       u'anime_side_story', u'anime_adaptation', u'anime_summary',
       u'anime_full_story', u'anime_parent_story', u'anime_sequel',
       u'anime_prequel', u'anime_alternative_setting', u'anime_spinoff',
       u'anime_other', u'review_timestamp', u'review_date', u'review_helpful',
       u'review_episodes_seen', u'review_overall', u'review_story',
       u'review_animation', u'review_sound', u'review_char

In [4]:
# df["essay{}".format(i)] = df["essay{}".format(i)].apply(lambda x: str(x))

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

In [6]:
def preprocessed(s):
    return lambda s: re.sub(r'(\d[\d\.])+', 'NUM', s.lower())
    

In [7]:
# v0 = TfidfVectorizer(stop_words="english", preprocessor=lambda x: re.sub(r'(\d[\d\.])+', 'NUM', x.lower()), min_df=10, max_df = 0.95, max_features =200, ngram_range=(1,2))
# v0.fit(df[df.anime_english_title == "Naruto"]["review_text"].values.astype('U'))
# v0.get_feature_names()


In [8]:
english_plus = [
    'a',
    'about',
    'above',
    'after',
    'again',
    'against',
    'all',
    'am',
    'an',
    'and',
    'any',
    'are',
    "aren't",
    'as',
    'at',
    'be',
    'because',
    'been',
    'before',
    'being',
    'below',
    'between',
    'both',
    'but',
    'by',
    "can't",
    'cannot',
    'could',
    "couldn't",
    'did',
    "didn't",
    'do',
    'does',
    "doesn't",
    'doing',
    "don't",
    'down',
    'during',
    'each',
    'few',
    'for',
    'from',
    'further',
    'had',
    "hadn't",
    'has',
    "hasn't",
    'have',
    "haven't",
    'having',
    'he',
    "he'd",
    "he'll",
    "he's",
    'her',
    'here',
    "here's",
    'hers',
    'herself',
    'him',
    'himself',
    'his',
    'how',
    "how's",
    'i',
    "i'd",
    "i'll",
    "i'm",
    "i've",
    'if',
    'in',
    'into',
    'is',
    "isn't",
    'it',
    "it's",
    'its',
    'itself',
    "let's",
    'me',
    'more',
    'most',
    "mustn't",
    'my',
    'myself',
    'no',
    'nor',
    'not',
    'of',
    'off',
    'on',
    'once',
    'only',
    'or',
    'other',
    'ought',
    'our',
    'ours',
    'ourselves',
    'out',
    'over',
    'own',
    'same',
    "shan't",
    'she',
    "she'd",
    "she'll",
    "she's",
    'should',
    "shouldn't",
    'so',
    'some',
    'such',
    'than',
    'that',
    "that's",
    'the',
    'their',
    'theirs',
    'them',
    'themselves',
    'then',
    'there',
    "there's",
    'these',
    'they',
    "they'd",
    "they'll",
    "they're",
    "they've",
    'this',
    'those',
    'through',
    'to',
    'too',
    'under',
    'until',
    'up',
    'very',
    'was',
    "wasn't",
    'we',
    "we'd",
    "we'll",
    "we're",
    "we've",
    'were',
    "weren't",
    'what',
    "what's",
    'when',
    "when's",
    'where',
    "where's",
    'which',
    'while',
    'who',
    "who's",
    'whom',
    'why',
    "why's",
    'with',
    "won't",
    'would',
    "wouldn't",
    'you',
    "you'd",
    "you'll",
    "you're",
    "you've",
    'your',
    'yours',
    'yourself',
    'yourselves',
    'zero'
]

In [9]:
english_plus2 = english_plus + ["episodes", "art", "character", "anime", "series", "watched", "watch", "num", "num0", "NUM", "NUM0"]
print(english_plus2)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

In [10]:
v0 = TfidfVectorizer(stop_words=english_plus2, preprocessor=lambda x: re.sub(r'(\d[\d\.])+', 'NUM', x.lower()), min_df=10, max_df = 0.95, max_features =200, ngram_range=(1,3))
v0.fit(df[df.anime_english_title == "Naruto"]["review_text"])
v0.get_feature_names()

[u'action',
 u'actually',
 u'almost',
 u'also',
 u'although',
 u'always',
 u'amazing',
 u'animation',
 u'animes',
 u'annoying',
 u'another',
 u'anything',
 u'arc',
 u'arcs',
 u'around',
 u'awesome',
 u'back',
 u'background',
 u'bad',
 u'battle',
 u'become',
 u'best',
 u'better',
 u'bit',
 u'bleach',
 u'boring',
 u'boy',
 u'can',
 u'cast',
 u'characters',
 u'completely',
 u'cool',
 u'development',
 u'didn',
 u'different',
 u'doesn',
 u'don',
 u'done',
 u'due',
 u'either',
 u'end',
 u'enjoy',
 u'enjoyable',
 u'enjoyed',
 u'enjoyment',
 u'enough',
 u'entire',
 u'episode',
 u'especially',
 u'even',
 u'even though',
 u'ever',
 u'every',
 u'everyone',
 u'everything',
 u'fact',
 u'fans',
 u'far',
 u'favorite',
 u'feel',
 u'fight',
 u'fighting',
 u'fights',
 u'filler',
 u'fillers',
 u'find',
 u'first',
 u'friends',
 u'fun',
 u'gaara',
 u'get',
 u'gets',
 u'give',
 u'go',
 u'going',
 u'good',
 u'got',
 u'great',
 u'hard',
 u'hate',
 u'hokage',
 u'however',
 u'interesting',
 u'isn',
 u'just',
 u

In [11]:
v1 = v0.fit_transform(df[df.anime_english_title == "Naruto"]["review_text"])
# print(v1.shape)
# vsum = (counts.toarray().sum(axis=0))
# print(vsum.shape)
vect_dict = v0.vocabulary_ #ohh these are just indexes cuz vector is always the same cuz vector space model
# print(len(vect_dict))
# print(vect_dict)
# newvect_dict = {}
# for key, value in vect_dict.items():
#     newvect_dict[key] = vsum[value]
# sorted_top = sorted(newvect_dict.items(), key=operator.itemgetter(1), reverse=True)
# print(sorted_top)
print(vect_dict)


{u'want': 189, u'much': 112, u'didn': 33, u'makes': 105, u'become': 20, u'problem': 137, u'worth': 197, u'another': 10, u'battle': 19, u'still': 168, u'naruto': 114, u'ninja': 119, u'special': 164, u'times': 180, u'boy': 26, u'first': 66, u'get': 70, u'point': 132, u'entire': 46, u'may': 108, u'like': 94, u'cast': 28, u'story': 169, u'kind': 87, u'also': 3, u'long': 98, u'new': 118, u'best': 21, u'life': 93, u'arcs': 13, u'take': 171, u'see': 152, u'anything': 11, u'moments': 111, u'shows': 157, u'real': 141, u'since': 158, u'pretty': 135, u'liked': 95, u'used': 185, u'doesn': 35, u'go': 73, u'one': 124, u'never': 117, u'give': 72, u'isn': 83, u'skip': 159, u'maybe': 109, u'thing': 172, u'throughout': 178, u'every': 52, u'end': 40, u'got': 76, u'ninjas': 120, u'naruto one': 115, u'something': 160, u'boring': 25, u'awesome': 15, u'around': 14, u'start': 165, u'scenes': 151, u'look': 99, u'started': 166, u'completely': 30, u'well': 192, u'kid': 86, u'going': 74, u'think': 174, u'show': 1

/Users/garygao/Desktop/CS4300_Flask_template/venv/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [12]:
cvect = CountVectorizer(stop_words=english_plus2, min_df=20, max_df = 0.75, max_features =200, ngram_range=(2,2))
counts = cvect.fit_transform(df[df.anime_english_title == "Naruto"]["review_text"])
# normalized_counts = normalize(counts, norm='l1', axis=1)
# cv_fit=cv.fit_transform(texts)

In [13]:
print(cvect.get_feature_names())
# print(counts.toarray())
print(counts.toarray().sum(axis=0))


[u'become hokage', u'can say', u'characters naruto', u'don like', u'dragon ball', u'enjoyment 10', u'even though', u'fight scenes', u'like naruto', u'lot people', u'main characters', u'many characters', u'naruto great', u'naruto one', u'naruto sasuke', u'naruto shippuden', u'one best', u'one piece', u'overall 10', u'pretty good', u'pretty much', u'really good', u'rock lee', u'sound 10', u'story 10', u'story naruto', u've seen', u'voice acting', u'watching naruto']
[22 27 35 26 32 26 56 38 30 25 49 23 22 56 33 37 25 47 26 41 22 34 35 29
 31 31 29 23 31]


In [14]:
counts_sum = (counts.toarray().sum(axis=0)).tolist()
cvect_dict = cvect.vocabulary_ #ohh these are just indexes cuz vector is always the same cuz vector space model
newcvect_dict = {}
for key, value in cvect_dict.items():
    newcvect_dict[key] = counts_sum[value]
sorted_top = sorted(newcvect_dict.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_top)

[(u'even though', 56L), (u'naruto one', 56L), (u'main characters', 49L), (u'one piece', 47L), (u'pretty good', 41L), (u'fight scenes', 38L), (u'naruto shippuden', 37L), (u'characters naruto', 35L), (u'rock lee', 35L), (u'really good', 34L), (u'naruto sasuke', 33L), (u'dragon ball', 32L), (u'story 10', 31L), (u'watching naruto', 31L), (u'story naruto', 31L), (u'like naruto', 30L), (u've seen', 29L), (u'sound 10', 29L), (u'can say', 27L), (u'overall 10', 26L), (u'enjoyment 10', 26L), (u'don like', 26L), (u'one best', 25L), (u'lot people', 25L), (u'voice acting', 23L), (u'many characters', 23L), (u'become hokage', 22L), (u'naruto great', 22L), (u'pretty much', 22L)]


In [15]:
cvect = CountVectorizer(stop_words=english_plus2, min_df=20, max_df = 0.75, max_features =200, ngram_range=(1,2))
X = cvect.fit_transform(df[df.anime_english_title == "Naruto"]["review_text"])
terms = cvect.get_feature_names()
len(terms)


200

In [16]:
def getcollocations_matrix(X):
    XX=X.T.dot(X)  ## multiply X with it's transpose to get number docs in which both w1 (row) and w2 (column) occur
    term_freqs = np.asarray(X.sum(axis=0)) ## number of docs in which a word occurs
    pmi = XX.toarray() * 1.0  ## Casting to float, making it an array to use simple operations
    pmi /= term_freqs.T ## dividing by the number of documents in which w1 occurs
    pmi /= term_freqs  ## dividing by the number of documents in which w2 occurs
    
    return pmi  # this is not technically PMI beacuse we are ignoring some normalization factor and not taking the log 
                # but it's sufficient for ranking

In [17]:

pmi_matrix = getcollocations_matrix(X)

def getcollocations(w,PMI_MATRIX=pmi_matrix,TERMS=terms):
    if w not in TERMS:
        return []
    idx = TERMS.index(w)
    col = PMI_MATRIX[:,idx].ravel().tolist()
    return sorted([(TERMS[i],val) for i,val in enumerate(col)],key=operator.itemgetter(1),reverse=True)

In [18]:
# getcollocations("good")

tokenizer = TweetTokenizer()

In [19]:
reviews_pos_tagged=[pos_tag(tokenizer.tokenize(m)) for m in df[df.anime_english_title == "Naruto"]["review_text"]]

## Reconstructing adjective-and-adverb-only reviews
reviews_adj_adv_only=[" ".join([w for w,tag in m if tag in ["JJ","RB","RBS","RBJ","JJR","JJS"]])
                      for m in reviews_pos_tagged]

In [20]:
X = cvect.fit_transform(reviews_adj_adv_only)
terms = cvect.get_feature_names()
pmi_matrix=getcollocations_matrix(X)
getcollocations("good",pmi_matrix,terms)

[(u'pretty good', 0.009434968017057569),
 (u'just good', 0.008833384099908621),
 (u'good', 0.008624256118130032),
 (u'especially', 0.007357248295414607),
 (u'worst', 0.007249466950959488),
 (u'really good', 0.007176660252743252),
 (u'stupid', 0.007107320540156362),
 (u'better', 0.007045517752850654),
 (u'instead', 0.007023705004389816),
 (u'decent', 0.00698478053084332),
 (u'nice', 0.0069409789955995095),
 (u'hard', 0.006860109390933531),
 (u'pretty', 0.006837732519667672),
 (u'funny', 0.006823027718550107),
 (u'already', 0.006809271614278836),
 (u'finally', 0.0066906845084920225),
 (u'entire', 0.00648020402190727),
 (u'easy', 0.006472738349070971),
 (u'don', 0.006396588486140725),
 (u'least', 0.006396588486140725),
 (u'sad', 0.006396588486140725),
 (u'mostly', 0.0063278079647843734),
 (u'isn', 0.006323064480552901),
 (u'never', 0.006283374530633809),
 (u'sometimes', 0.0062823636917453555),
 (u'interesting', 0.006269672047923647),
 (u'easily', 0.006263326226012793),
 (u'deep', 0.006238

In [21]:
def seed_score(pos_seed,PMI_MATRIX=pmi_matrix,TERMS=terms):
    score=defaultdict(int)
    for seed in pos_seed:
        c=dict(getcollocations(seed,PMI_MATRIX,TERMS))
        for w in c:
            score[w]+=c[w]
    return score

In [22]:
sorted(seed_score(['good','great','perfect','cool']).items(),key=operator.itemgetter(1),reverse=True)

[(u'perfect', 0.05406024281785207),
 (u'cool', 0.04130050982628741),
 (u'stupid', 0.03540093192711432),
 (u'single', 0.031315967798042604),
 (u'big', 0.03127340465236234),
 (u'almost', 0.030247440373082305),
 (u'favorite', 0.029348423300345948),
 (u'japanese', 0.028702039429128433),
 (u'definitely', 0.02826773503337741),
 (u'nice', 0.027931485395890132),
 (u'useless', 0.02786024093367844),
 (u'awesome', 0.02783096732665686),
 (u'great', 0.027700078850499087),
 (u'extremely', 0.027307147381750267),
 (u'entire', 0.026943146222204117),
 (u'important', 0.026538627416519748),
 (u'memorable', 0.02645585239688408),
 (u'll', 0.026170093220540286),
 (u'due', 0.02609144410107902),
 (u'finally', 0.026055409520173323),
 (u'boring', 0.025953748456804326),
 (u'worst', 0.02592879231854766),
 (u'beautiful', 0.025448186718010038),
 (u'especially', 0.02541002631471728),
 (u'instead', 0.025144840923920844),
 (u'best ever', 0.025140247461660666),
 (u'enough', 0.024907235710225756),
 (u'don', 0.02481188689

In [26]:
posscores=seed_score(['good','great','perfect','cool', "amazing", "enjoyable", "favorite", "worth", "greatest", "awesome", "beautiful", "deep", "unique", "nice"])
negscores=seed_score(['bad','terrible','wrong',"crap","long","boring", "stupid", "worst", "slow", "useless", "filler"])

## sentiment polarity score will be the difference between the words that are close to the positive seed
## and the words that are close to the negative seed
sentscores={}
for w in terms:
    sentscores[w] = posscores[w] - negscores[w]
    

In [27]:
sorted(sentscores.items(),key=operator.itemgetter(1),reverse=False)

[(u'powerful', -0.015466895938554728),
 (u'stupid', -0.01522393356094881),
 (u'worst', -0.015026205025805373),
 (u'boring', -0.010692263758317083),
 (u'decent', -0.010612664903235487),
 (u'slow', -0.009209735141256642),
 (u'wrong', -0.008537953803513756),
 (u'else', -0.005711983882292326),
 (u'useless', -0.0033606961558660386),
 (u'easily', -0.001019587118201673),
 (u'actually', -0.0005885237431715595),
 (u'least', 0.0031956724056914007),
 (u'usually', 0.003657647027155028),
 (u'average', 0.004493089182391566),
 (u'rather', 0.005643309912511746),
 (u'bad', 0.00619824031146695),
 (u'yet', 0.00652583339895392),
 (u'ninjas', 0.00672293486787124),
 (u'naruto', 0.007032726738436114),
 (u'single', 0.00820569019305896),
 (u'quite', 0.008481600545924159),
 (u'certain', 0.010316374951532412),
 (u'really really', 0.010859441959668405),
 (u'super', 0.011194176737043284),
 (u'little', 0.011343038507894261),
 (u'stronger', 0.01351325144880594),
 (u'simple', 0.01374781791088165),
 (u'completely', 0.

In [31]:
cvect = CountVectorizer(stop_words=english_plus2, min_df=20, max_df = 0.75, max_features =200, ngram_range=(1,2))
X = cvect.fit_transform(df[df.anime_english_title == "Bleach"]["review_text"])
terms = cvect.get_feature_names()
len(terms)
pmi_matrix = getcollocations_matrix(X)
reviews_pos_tagged=[pos_tag(tokenizer.tokenize(m)) for m in df[df.anime_english_title == "Bleach"]["review_text"]]

## Reconstructing adjective-and-adverb-only reviews
reviews_adj_adv_only=[" ".join([w for w,tag in m if tag in ["JJ","RB","RBS","RBJ","JJR","JJS"]])
                      for m in reviews_pos_tagged]

X = cvect.fit_transform(reviews_adj_adv_only)
terms = cvect.get_feature_names()
pmi_matrix=getcollocations_matrix(X)
posscores=seed_score(['good','great','perfect','cool', "amazing", "enjoyable", "favorite", "worth", "greatest", "awesome", "beautiful", "funny"])
negscores=seed_score(['bad','terrible','wrong',"crap","long","boring", "stupid", "worst", "slow", "useless", "old", "terrible"])

## sentiment polarity score will be the difference between the words that are close to the positive seed
## and the words that are close to the negative seed
sentscores={}
for w in terms:
    sentscores[w] = posscores[w] - negscores[w]
    
sorted(sentscores.items(),key=operator.itemgetter(1),reverse=False)

[(u'worst', -0.030854972591531887),
 (u'slow', -0.027621357584242084),
 (u'powerful', -0.02740695589289833),
 (u'else', -0.022390487568882254),
 (u'stupid', -0.022279546861475166),
 (u'boring', -0.021547157295288014),
 (u'quite', -0.0214276726585641),
 (u'useless', -0.018231471201610075),
 (u'yet', -0.01784097831524957),
 (u'rather', -0.017432290568039313),
 (u'certain', -0.016482651921486326),
 (u'decent', -0.01551969820018037),
 (u'wrong', -0.01551819811482811),
 (u'usually', -0.015357127248057843),
 (u'easily', -0.012534056637665637),
 (u'simple', -0.011383649770530545),
 (u'actually', -0.010999874221114703),
 (u'least', -0.009135901733398205),
 (u'old', -0.007956129944711218),
 (u'mostly', -0.007024692266263788),
 (u'single', -0.0068007911984426656),
 (u'bad', -0.0060609698663795855),
 (u'often', -0.005940208358494653),
 (u'stronger', -0.004237464828451734),
 (u'completely', -0.003996418903725277),
 (u'little', -0.0037207327600945472),
 (u'deep', -0.0007030843919536706),
 (u'soon',